In [ ]:
# %% [markdown]
# # Promotional Pricing Impact Analysis
# 
# Evaluates the revenue impact of proposed promotional pricing.

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.data import make_dataset

# %%
# Load data
df, df_promo = make_dataset.load_raw_data()
results_df = pd.read_csv('reports/results/model_results.csv')

# %%
# Prepare promotional data
df_promo = df_promo.melt(
    id_vars=['Unnamed: 0'], 
    var_name='Date', 
    value_name='UnitPrice'
).rename(columns={'Unnamed: 0':'StockCode'})

df_promo['Date'] = pd.to_datetime(df_promo['Date'])
df_promo['LastDate'] = df_promo['Date'] + pd.DateOffset(years=-1)
df_promo['LastDate'] = df_promo['LastDate'].dt.to_period('D')

# %%
# Merge with elasticity estimates
df_promo['StockCode'] = df_promo['StockCode'].astype(str)
results_df['StockCode'] = results_df['StockCode'].astype(str)
df_promo = df_promo.merge(results_df[['StockCode','Elasticity']], on='StockCode')

# %%
# Prepare historical data for comparison
df_full = df.groupby(['StockCode','InvoiceDate']).agg(
    TotalQuantity=('Quantity', 'sum'),
    AvgPrice=('UnitPrice', 'mean')
).reset_index()
df_full['StockCode'] = df_full['StockCode'].astype(str)

# %%
# Merge with historical data
df_promo = df_promo.merge(
    df_full[['StockCode','InvoiceDate','AvgPrice','TotalQuantity']], 
    left_on=['StockCode', 'LastDate'], 
    right_on=['StockCode', 'InvoiceDate'], 
    how='left'
)

# %%
# Handle missing values
df_promo['InvoiceDate'] = np.where(df_promo['UnitPrice'] == 0, df_promo['LastDate'], df_promo['InvoiceDate'])
df_promo['AvgPrice'] = np.where(df_promo['UnitPrice'] == 0, 0, df_promo['AvgPrice'])
df_promo['TotalQuantity'] = np.where(df_promo['UnitPrice'] == 0, 0, df_promo['TotalQuantity'])

df_promo['InvoiceDate'] = df_promo['InvoiceDate'].fillna(df_promo['LastDate'])
df_promo['AvgPrice'] = df_promo['AvgPrice'].fillna(df_promo['AvgPrice'].median())
df_promo['TotalQuantity'] = df_promo['TotalQuantity'].fillna(df_promo['TotalQuantity'].median())

# %%
# Rename columns
df_promo.rename(columns={
    'TotalQuantity': 'PriorSales',
    'AvgPrice': 'PriorPrice',
    'UnitPrice': 'ProposedPrice',
    'Date':'PriceDate'
}, inplace=True)

# %%
# Calculate forecasted demand and revenue
df_promo['PriorRevenue'] = df_promo['PriorPrice'] * df_promo['PriorSales'] 
df_promo['ForecastedSales'] = ((df_promo['PriorSales']) * 
                              (df_promo['ProposedPrice'].replace(0,np.nan) / 
                               df_promo['PriorPrice'].replace(0,np.nan)) ** 
                              df_promo['Elasticity']).fillna(0)
df_promo['ForecastedRevenue'] = df_promo['ProposedPrice'] * df_promo['ForecastedSales']

# %%
# Save results
df_promo.to_csv('reports/results/promo_analysis_results.csv', index=False)

# %%
# Revenue comparison by SKU
revenue_comparison = df_promo.groupby('StockCode')[['PriorRevenue','ForecastedRevenue']].sum()
revenue_comparison['Change'] = revenue_comparison['ForecastedRevenue'] - revenue_comparison['PriorRevenue']
revenue_comparison['Pct_Change'] = (revenue_comparison['Change']/revenue_comparison['PriorRevenue'])*100

plt.figure(figsize=(12,6))
revenue_comparison['Pct_Change'].sort_values().plot(kind='barh', color="#135AE8D9")
plt.title('Percentage Revenue Change by SKU')
plt.xlabel('Percentage Change (%)')
plt.ylabel('SKU')
plt.axvline(0, color='black')
plt.tight_layout()
plt.savefig('reports/figures/revenue_change_by_sku.png')
plt.show()

# %%
# Total impact analysis
total_prior = df_promo['PriorRevenue'].sum()
total_forecast = df_promo['ForecastedRevenue'].sum()

print(f"Total Revenue Lift: ${(total_forecast - total_prior):,.2f}")
print(f"Percentage Lift: {((total_forecast - total_prior)/total_prior)*100:.2f}%")

# %%
# Plot total impact
summary = pd.DataFrame({
    'Period': ['Current', 'Forecasted'],
    'Revenue': [total_prior, total_forecast]
})

plt.figure(figsize=(8, 5))
sns.barplot(data=summary, x='Period', y='Revenue', palette=["#135AE8D9", "#F8D700"])
plt.title(f'Pricing Plan Impact\nTotal Lift: ${(total_forecast-total_prior):,.2f} ({((total_forecast-total_prior)/total_prior)*100:.1f}%)')
plt.ylabel('Total Revenue ($)')
plt.savefig('reports/figures/total_revenue_impact.png')
plt.show()